## Approach Ln

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy.stats import poisson
from scipy.stats import poisson
from scipy.stats import chisquare

In [ ]:
compras = pd.read_excel('informacion_compra.xlsx')

In [ ]:
compras

In [ ]:
facturas = []
productos = []
for _,i in compras.iterrows():
    facturas += [i[2]]*i[1]
for _,i in compras.iterrows():
    productos += [i[0]]*i[1]

compras = pd.DataFrame({
    "Factura": facturas,
    "Producto": productos
})

In [ ]:
df = compras.groupby(['Factura']).size().reset_index(name='Frequency').Frequency.value_counts()
df = df.reset_index()
df.columns = ['Pedidos', 'Frecuencia']
df['Frecuencia ln'] = np.log(df['Frecuencia'])
valor_medio = np.sum(df['Pedidos'] * df['Frecuencia ln']) / np.sum(df['Frecuencia ln'])
df['Poisson'] = poisson.pmf(df['Pedidos'], valor_medio)
df['valores_esperados'] = df['Poisson'] * np.sum(df['Frecuencia ln'])
df

In [ ]:
# Set the style of the plot
sns.set(style="whitegrid")

# Create a figure and axis
plt.figure(figsize=(8, 5))

# Plot the first set of bars with higher transparency
sns.barplot(x='Pedidos', y='Poisson', data=df, alpha=0.6, label='Datos Distribuidos en Poisson')

#sns.barplot(x='Pedidos', y='Frecuencia', data=df, alpha=0.6, label = 'Original')

# Plot the second set of bars with lower transparency
#sns.barplot(x='Pedidos', y='Frecuencia ln', data=df, alpha=0.3, label='Valores')

# Add labels and title
plt.xlabel('Pedidos')
plt.ylabel('ln(Conteo)')
plt.title('Distribución poisson')

# Show legend
plt.legend()

# Show the plot
plt.show()


### Distribución de productos

In [ ]:
df2 = compras.groupby(['Producto']).size().reset_index(name='Frequency').Frequency.value_counts()
df2 = df2.reset_index()
df2.columns = ['Producto', 'Frecuencia']
df2['Frecuencia ln'] = np.log(df2['Frecuencia'])
valor_medio = np.sum(df2['Producto'] * df2['Frecuencia ln']) / np.sum(df2['Frecuencia ln'])
df2['Poisson'] = poisson.pmf(df2['Producto'], valor_medio)
df2['valores_esperados'] = df2['Poisson'] * np.sum(df2['Frecuencia ln'])
display(df2)

In [ ]:
sns.set(style="whitegrid")

# Create a figure and axis
plt.figure(figsize=(8, 5))

# Plot the first set of bars with higher transparency
sns.barplot(x='Producto', y='Poisson', data=df2, alpha=0.6, label='Datos Distribuidos en Poisson')

#sns.barplot(x='Pedidos', y='Frecuencia', data=df, alpha=0.6, label = 'Original')

# Plot the second set of bars with lower transparency
#sns.barplot(x='Pedidos', y='Frecuencia ln', data=df, alpha=0.3, label='Valores')

# Add labels and title
plt.xlabel('Pedidos')
plt.ylabel('ln(Conteo)')
plt.title('Distribución poisson')

# Show legend
plt.legend()

# Show the plot
plt.show()

## Generación de pedidos

In [ ]:
def montecarlo(df, n):  
    resultados = []
    for _ in range(n):
        num = np.random.rand()
        for j in range(len(df)):
            if df[j][3] <= num < df[j][4]:
                resultados.append(j)
            
    return resultados

In [ ]:
pedidos = df[['Pedidos','Poisson']]
pedidos['Acumulado'] = pedidos.Poisson.cumsum()
pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]
pedidos['Superior'] = pedidos['Acumulado']
pedidos = pedidos.to_numpy()

In [ ]:
num_pedidos = montecarlo(pedidos,df.Frecuencia.sum())
#num_pedidos = montecarlo(pedidos,compras.size)


In [ ]:
producto = df2[['Producto','Poisson']]
producto['Acumulado'] = producto.Poisson.cumsum()
producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]
producto['Superior'] = producto['Acumulado']
producto = producto.to_numpy()

In [ ]:
envios = [np.array(montecarlo(producto, num)) for num in montecarlo(pedidos,df.Frecuencia.sum()) if num!=0]
envios_np = envios

In [ ]:
enviosArreglo = np.array([[-1,-1,-1]])
for i in range(len(envios)):
    a = np.unique(np.array(envios[i]), return_counts=True)
    enviosArreglo = np.concatenate((enviosArreglo, np.array([np.array([i]*len(a[0])),a[0],a[1]]).T))
enviosArreglo

In [ ]:
envios = pd.DataFrame(enviosArreglo[1:], columns = ['Factura', 'Producto', 'Unidades'])

## Prueba Con Gráficas

In [ ]:
df3 = envios.groupby(['Factura']).size().reset_index(name='Frequency').Frequency.value_counts()
df3 = df3.reset_index()
df3.columns = ['Pedidos', 'Frecuencia']
valor_medio = np.sum(df3['Pedidos'] * df3['Frecuencia']) / np.sum(df3['Frecuencia'])
df3['Poisson'] = poisson.pmf(df3['Pedidos'], valor_medio)
df3['valores_esperados'] = df3['Poisson'] * np.sum(df3['Frecuencia'])
df3

In [ ]:
sum((list(df3.sort_values(by="Pedidos").Poisson-df.sort_values(by="Pedidos").Poisson))[:16])/16

In [ ]:
# Set the style of the plot
sns.set(style="whitegrid")

# Create a figure and axis
plt.figure(figsize=(8, 5))

# Plot the first set of bars with higher transparency
sns.barplot(x='Pedidos', y='Poisson', data=df, alpha=0.6, label='Original',color='darkorange')

sns.barplot(x='Pedidos', y='Poisson', data=df3, alpha=0.6, label = 'Simulación',color='steelblue')

# Plot the second set of bars with lower transparency
#sns.barplot(x='Pedidos', y='Frecuencia ln', data=df, alpha=0.3, label='Valores')

# Add labels and title
plt.xlabel('Pedidos')
plt.ylabel('Probabilidad')
plt.title('Distribución de probabilidad')

# Show legend
plt.legend()

# Show the plot
plt.show()

## Costo de envíos

In [ ]:
dp = pd.read_csv('info_productos.csv')
dp.columns = ['Producto', 'Volumen']
nuevo_registro = {"Producto": 0, "Volumen": 0}
dp = pd.concat([pd.DataFrame([nuevo_registro]), dp], ignore_index=True)
volumenes = dp.to_numpy()[:,1]

In [ ]:
dimensiones = []
for pedido in envios_np:
    acumulado = 0
    for i in pedido:
        acumulado += volumenes[i]
    dimensiones.append(acumulado)

In [ ]:
dimensiones

In [ ]:
display(envios.groupby('Factura'))